# Requirements

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00


In [4]:
!pip install pandas

In [5]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.7 MB/s eta 0:00:00


In [6]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00


# Imports

In [94]:
import os
import warnings
import numpy as np
import openai
import evaluate
from google.colab import drive
from datasets import load_dataset, Features, Value, ClassLabel
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline,
    TextClassificationPipeline,
)

# Env setup

In [8]:
os.environ['OPENAI_API_KEY'] = 'your-key-here'
openai.api_key = os.getenv("OPENAI_API_KEY")

# Text classifier

## Load and fine-tune BERT

In [176]:
warnings.filterwarnings('ignore')

In [167]:
drive.mount('/content/drive', force_remount=True)
features = Features({"text":Value('string'), "label":ClassLabel(names=['TRUE', 'FALSE'])})
dataset = load_dataset(
    "csv",
    data_files="/content/drive/MyDrive/challenge/HalQuestions_StatusOnly_Test.csv",
    sep=",",
    names=["text", "label"],
    features=features,
)

Mounted at /content/drive


In [168]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [169]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased",
    num_labels=2,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [170]:
metric = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    scores = np.max(logits, axis=-1)
    return metric.compute(prediction_scores=scores, references=labels)

In [171]:
train_test_ds = tokenized_datasets['train'].train_test_split(test_size=0.2)

In [172]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    num_train_epochs=5,
)

In [173]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_ds['train'],
    eval_dataset=train_test_ds['test'],
    compute_metrics=compute_metrics,
)

Evaluate the base model (that is, before fine-tuning):

In [174]:
trainer.evaluate()

{'eval_loss': 0.9321605563163757,
 'eval_roc_auc': 0.4588293650793651,
 'eval_runtime': 7.6838,
 'eval_samples_per_second': 27.721,
 'eval_steps_per_second': 3.514}

Note the current roc auc is rather low.
Now fine-tune the model

In [175]:
trainer.train()

Epoch,Training Loss,Validation Loss,Roc Auc
1,No log,0.369411,0.929067
2,No log,0.156992,0.932788
3,No log,0.314399,0.901290
4,No log,0.230464,0.918651
5,0.144600,0.241032,0.926835


TrainOutput(global_step=535, training_loss=0.13791261721994275, metrics={'train_runtime': 457.9109, 'train_samples_per_second': 9.281, 'train_steps_per_second': 1.168, 'total_flos': 1118221985280000.0, 'train_loss': 0.13791261721994275, 'epoch': 5.0})

Having more than 1 epoch doesn't seem to have helped much.
But, nevertheless, note the roc auc is already much better after the first epoch (compared to the base model), indicating the fine-tuning helped.

## Manually check a few cases

Manually try the model on a few questions.

`lABEL_1` means *cannot answer*

`LABEL_0` means *can answer*.

In [183]:
pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    device='cuda:0',
)

In [184]:
pipe('Can I apply for a loan if I have a low credit score?')

[{'label': 'LABEL_1', 'score': 0.9996961355209351}]

In [185]:
pipe('Is my loan approved?')

[{'label': 'LABEL_0', 'score': 0.9928457140922546}]

In [186]:
pipe('What is the current status of my loan?')

[{'label': 'LABEL_0', 'score': 0.9936569333076477}]

In [187]:
pipe('Do I need to provide proof of residence for the loan application?')

[{'label': 'LABEL_1', 'score': 0.9996516704559326}]

In [188]:
pipe('How long is the repayment period for this loan?')

[{'label': 'LABEL_0', 'score': 0.9926026463508606}]

In [189]:
pipe('What is the term for this loan?')

[{'label': 'LABEL_1', 'score': 0.9773424863815308}]

In [190]:
pipe("How do I know the total amount I will repay over the loan's tenure?")

[{'label': 'LABEL_0', 'score': 0.7875183820724487}]

In [191]:
pipe("How do I know how much I will end up paying after the tenure?")

[{'label': 'LABEL_1', 'score': 0.583543062210083}]

In [192]:
pipe("Who is the president of the United States?") # :(

[{'label': 'LABEL_0', 'score': 0.9439690113067627}]

In [193]:
pipe("Is this interest rate convenient for me?")

[{'label': 'LABEL_1', 'score': 0.9428815841674805}]

In [194]:
pipe("What happens in the case of loan default?")

[{'label': 'LABEL_1', 'score': 0.9992445707321167}]

# LLM

## Functions and constants

In [195]:
def get_prompt(context, question):
  prompt = f"""
  Assume you are a polite assistant answer questions to a customer about a loan application
  Follow exactly these 2 steps:
  1. Read the context below
  Context : {context}
  2. Answer the question using only this context
  User Question: {question}

  If you don't have any context and are unsure of the answer, politely reply that you don't know about this topic and suggest that a more senior lending officer may need to answer that question.
  """
  return prompt


In [196]:
def ask(context, question):
  response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': get_prompt(context, question)}],
    temperature=0,
    max_tokens=1024
  )
  return response['choices'][0]['message']['content']

In [197]:
context_counter = """
  The loan application is at the initial stage. It's currently at the counter.
  The rate for the loan is 3.75.
  The loan amount is 18000.
  The loan term is 60 days.
  The estimated payment is 361.65
"""

In [198]:
context = context_counter

## Test cases

Most of the questions extracted from the provided *HalQuestions_StatusOnly_Test* document. I added a few examples I considered interesting.

In [199]:
question = 'What is the current status of my loan application?'
ask(context, question)

'The current status of your loan application is that it is at the initial stage and is currently at the counter.'

Sounds a bit robotic, but the answer is generally correct.

In [200]:
question = 'When can I expect a decision on my loan application?'
ask(context, question)

'Based on the current stage of your loan application, it is difficult for me to provide an exact timeframe for when a decision will be made. However, the lending process typically involves a thorough evaluation of your application, which can take some time. I recommend reaching out to a senior lending officer who can provide you with more specific information regarding the timeline for a decision on your loan application.'

In [201]:
question = 'Can I check the status of my loan application online?'
ask(context, question)

"I'm sorry, but as a polite assistant, I don't have access to specific information about the status of your loan application. I recommend reaching out to a senior lending officer who can provide you with the most accurate and up-to-date information regarding your application."

In [202]:
question = 'Are there any updates on the progress of my loan application?'
ask(context, question)

"I apologize for any inconvenience, but as a polite assistant, I don't have access to real-time updates on the progress of your loan application. I recommend reaching out to a senior lending officer who would be better equipped to provide you with the most accurate and up-to-date information regarding your application."

In [203]:
question = 'What is the interest rate for this loan?'
ask(context, question)

'The interest rate for this loan is 3.75%.'

In [205]:
question = 'What is the interest rate on my loan?'
ask(context, question)

'The interest rate on your loan is 3.75%.'

In [207]:
question = 'Are there any hidden fees associated with this loan?'
ask(context, question)

'Based on the information provided, there is no mention of any hidden fees associated with this loan. However, for a more detailed and accurate answer, I recommend consulting with a senior lending officer who can provide you with more specific information regarding any potential fees.'

In [208]:
question = 'Is the rate of my payment convenient for me?'
ask(context, question)

'Based on the provided context, the rate for your loan payment is 3.75. Whether this rate is convenient for you depends on your personal financial situation and preferences. It would be best to evaluate your own financial capabilities and determine if this rate aligns with your needs and budget.'

A bit verbose. Also, it includes some information it has not been asked about (the rate). It sound reasonable overall though.

In [209]:
question = 'How long does it typically take to process a loan application?'
ask(context, question)

'Typically, the processing time for a loan application can vary depending on various factors. However, since the loan application is currently at the initial stage and at the counter, it is difficult to provide an exact timeframe. I recommend reaching out to a senior lending officer who would be able to provide you with more specific information regarding the processing time for your loan application.'

The first sentence sounds a bit weird. Overall, the answer in reasonable.

In [210]:
question = 'How long will it take to process my loan application?'
ask(context, question)

"Based on the current stage of your loan application, it is difficult for me to provide an exact timeframe for the processing. However, rest assured that our team is working diligently to process your application as quickly as possible. If you have any specific concerns or need further assistance, please let me know and I'll be happy to help."

In [211]:
question = 'Can I choose between fixed and variable interest rates for the loan?'
ask(context, question)

"I apologize for any confusion, but at this initial stage of the loan application, I don't have information about the options for fixed or variable interest rates. It would be best to consult with a senior lending officer who can provide more detailed information and assist you in choosing the most suitable interest rate option for your loan."

The first sentence is a bit creative. Overall, the answer is reasonable.

In [212]:
question = 'How long is the repayment period for this loan?'
ask(context, question)

'The repayment period for this loan is 60 days.'

In [213]:
question = 'How will I know if my loan application has been denied?'
ask(context, question)

'Once your loan application has been reviewed by our lending team, you will receive a notification regarding the status of your application. If your loan application has been denied, you will be informed about the decision and the reasons behind it. If you have any further questions or concerns, please feel free to ask.'

The first sentence is definitely too creative. However, the answer does seem to point the customer in the right direction.

In [214]:
question = 'How will I know if my loan application is missing any documents?'
ask(context, question)

'As a polite assistant, I can inform you that at the initial stage of your loan application, the lending officer at the counter will review your documents to ensure that all necessary paperwork is included. If any documents are missing, they will inform you and provide guidance on what is required.'

Again probably too creative. Not necessarily wrong though.

In [215]:
question = 'How is the loan interest calculated?'
ask(context, question)

'The loan interest is calculated based on the loan rate, which in this case is 3.75%. The interest is calculated on the loan amount of $18,000 over the loan term of 60 days. The specific calculation formula may vary depending on the terms and conditions of the loan agreement.'

Looks like a quite reasonable answer (not sure it it's helpful though).

In [216]:
question = 'How is the interest calculated on my loan?'
ask(context, question)

'The interest on your loan is calculated based on the loan amount, the interest rate, and the loan term. In this case, the interest rate is 3.75% and the loan amount is $18,000. The loan term is 60 days. The estimated payment of $361.65 includes both the principal amount and the interest.'

A bit more creative compared to the previuos answer, although it still looks reasonable at a glance.

In [217]:
question = 'What is the total amount I will repay over the loan term?'
ask(context, question)

'The total amount you will repay over the loan term is $21,699.'

This looks wrong...

In [218]:
question = 'How will I be notified of the loan application decision?'
ask(context, question)

'Once the loan application decision has been made, you will be notified through the contact information provided in your application. This could be through a phone call, email, or letter.'

Here it's clearly answering without considering the context at all (the contex provides no information about notifications). While it does not look particularly problematic in this case, it's definetily not the expected behavior.

In [219]:
question = 'How do I know if my loan has been approved?'
ask(context, question)

"Based on the information provided, I can inform you that as an initial stage, your loan application is currently at the counter. However, I don't have access to real-time updates on the approval status of your loan. To get accurate information regarding the approval of your loan, I recommend reaching out to a senior lending officer who will be able to provide you with the most up-to-date information."

In [220]:
question = 'What is the current stage of the loan application review process?'
ask(context, question)

'The current stage of the loan application review process is at the initial stage. It is currently at the counter.'

In [221]:
question = 'How long does it usually take to verify the submitted documents?'
ask(context, question)

'The verification process for submitted documents typically takes a few business days. However, please note that the exact timeframe may vary depending on the volume of applications and the complexity of the documents. If you have any specific concerns or need further assistance, I recommend reaching out to a senior lending officer who can provide more accurate information.'

The first 2 sentences are quite creative. However, the answer looks reasonable overall.

In [223]:
question = 'How will I receive the loan funds if my application is approved?'
ask(context, question)

'If your loan application is approved, you will receive the loan funds through a method that will be discussed with you by the lending officer. They will provide you with the necessary information on how the funds will be disbursed to you.'

Probably too creative...

In [224]:
question = 'How will I be notified of any changes in the refinancing process?'
ask(context, question)

"As a polite assistant, I can inform you that you will be notified of any changes in the refinancing process through the contact information provided in your loan application. The lending officer will reach out to you via phone or email to inform you about any updates or changes. If you have any specific concerns or questions regarding the refinancing process, please feel free to let me know, and I'll do my best to assist you."

Again very creative here, although reasonable answer in general.

In [225]:
question = 'How will I be notified if the request for a lower interest rate is approved?'
ask(context, question)

'As a polite assistant, I can inform you that at the initial stage of the loan application, the request for a lower interest rate has not been addressed yet. However, once a decision is made regarding the interest rate, you will be notified through the contact information provided in your application. If you have any further questions or concerns, please feel free to ask.'

Probably too creative...

In [226]:
question = 'When is my first payment due?'
ask(context, question)

'The first payment for your loan is typically due within 30 days of the loan being approved and disbursed. However, since your loan term is 60 days, it is possible that the first payment may be due within that timeframe. I recommend contacting a senior lending officer for more specific information regarding the due date of your first payment.'

Too creative. But at least it suggests contacting a senior lending officer.

In [227]:
question = 'What is the loan tenure?'
ask(context, question)

'The loan tenure is 60 days.'

In [229]:
question = 'How do I get a loan breakdown for different components?'
ask(context, question)

'To get a loan breakdown for different components, you can speak to a senior lending officer who will be able to provide you with more detailed information about the breakdown of the loan components. They will be able to assist you further in understanding the specific breakdown of the loan.'

In [230]:
question = "How do I get a document confirming my loan's interest rate?"
ask(context, question)

"To get a document confirming your loan's interest rate, you can request it from the lending officer at the counter where your loan application is currently being processed. They should be able to provide you with the necessary documentation."

In [231]:
question = 'When will I receive my credit card if approved?'
ask(context, question)

"As a polite assistant, I can provide information based on the given context. However, the context does not mention anything about a credit card. It only provides details about a loan application, such as the loan rate, amount, term, and estimated payment. Therefore, I'm unable to provide an answer regarding the timeline for receiving a credit card if approved. I recommend reaching out to a more senior lending officer who may have more information on this topic."

Too verbose (and wih unnecesary details), but generally correct.

In [232]:
question = 'Are there any discrepancies in my loan application?'
ask(context, question)

"Based on the information provided, there are no discrepancies in your loan application. The loan application is currently at the counter and the details such as the loan rate, amount, term, and estimated payment have been provided. If you have any specific concerns or questions regarding your loan application, please let me know and I'll be happy to assist you further."

A potentially problematic answer, because it assures there are no discrepancies, while the context does not specify whether there are discrepancies or not.

In [233]:
question = 'Who was the first president of the United States?'
ask(context, question)

"I apologize, but I don't have any information about the first president of the United States. My expertise is in assisting with loan applications. If you have any questions related to your loan application, I would be happy to help."

In [234]:
question = 'Is this a convinient loan for me?'
ask(context, question)

'Based on the provided context, I can say that the loan has a competitive interest rate of 3.75% and a loan term of 60 days. The estimated payment is $361.65. However, whether this loan is convenient for you depends on your individual financial situation and needs. It would be best to discuss your specific requirements with a senior lending officer who can provide personalized advice.'

In [235]:
question = 'Is a car faster than a plane?'
ask(context, question)

"As a polite assistant, I understand that you may have various questions, but I'm here to assist you specifically with your loan application. Regarding your question about whether a car is faster than a plane, I'm afraid I don't have any information on that topic. However, I would be more than happy to help you with any queries related to your loan application process. If you have any concerns or need further assistance, please feel free to ask."

## Conclusions

Overall, the LLM seems to do a reasonable work.

It usually sticks to the information provided in the contect and, eventhough it gets creative in several answers, it does not seem to misguide the customer or come up with crazy hallucinations.

That being said, it's definitely far from perfect and quite bad in some cases. For example, it seems to have problems with math (known issue), the answers sound quite robotic and it does come up with information not included in the context in ocassions. Further, the examples above can by no means be enough to say the model is robust in terms of jailbreaking.

# Loan assistant

In [236]:
def answer(question):
  answer_ = 'I recommend reaching out to a more senior lending officer who can help you with your inquiry.'
  can_answer = pipe(question)[0]['label'] =='LABEL_0'
  if can_answer:
    answer_ = ask(context, question)
  return answer_

## Test cases

In [237]:
answer('Is a car faster than a plane?')

'I recommend reaching out to a more senior lending officer who can help you with your inquiry.'

In [238]:
answer('Can I get a breakdown of my monthly payments?')

'Certainly! Based on the loan amount of $18,000, the loan term of 60 days, and the interest rate of 3.75%, your estimated monthly payment would be $361.65.'

In [239]:
answer('How long will it take to process my personal loan application?')

'The processing time for your personal loan application can vary depending on various factors. However, as your loan is currently at the initial stage and at the counter, it is best to check with a lending officer for a more accurate estimate of the processing time. They will be able to provide you with the most up-to-date information regarding the status of your application and the expected timeline.'

In [240]:
answer('What is the status of my personal loan application?')

'Based on the information provided, the status of your personal loan application is currently at the initial stage and it is at the counter.'

In [241]:
answer('Is a merchant marine card considered a valid form of ID?')

'I recommend reaching out to a more senior lending officer who can help you with your inquiry.'

In [242]:
answer('What is the tenure of my personal loan?')

'The tenure of your personal loan is 60 days.'

In [243]:
answer('How can I set up automatic repayments?')

'I recommend reaching out to a more senior lending officer who can help you with your inquiry.'

In [245]:
answer('When is my first loan repayment due?')

"The first loan repayment is typically due within a month after the loan is disbursed. However, since the loan application is still at the initial stage, I don't have access to the specific due date for your first repayment. I recommend reaching out to a senior lending officer who can provide you with more accurate information regarding the due date."

In [246]:
answer('How can I check the outstanding balance on my loan?')

'I recommend reaching out to a more senior lending officer who can help you with your inquiry.'

In [247]:
answer('Is the current status of my loan application something to be worried about?')

'Based on the current context, the loan application is at the initial stage and is currently at the counter. There is no information provided to suggest that there is any reason to be worried about the status of your loan application at this time. However, for more specific information and reassurance, it would be best to consult with a senior lending officer who can provide you with a more detailed update on your application.'

In [248]:
answer('Are there any penalties for early loan repayment?')

'I recommend reaching out to a more senior lending officer who can help you with your inquiry.'

In [249]:
answer('Who was the first president of the United States?')

"I apologize, but I don't have any information about the first president of the United States. My expertise is in assisting with loan applications. If you have any questions related to your loan application, I would be happy to help."

In [250]:
answer('Is a car faster than a plane?')

'I recommend reaching out to a more senior lending officer who can help you with your inquiry.'